In [1]:
#default_exp model.RT

In [2]:
#hide
%reload_ext autoreload
%autoreload 2

In [3]:
#export
import torch
import pandas as pd
import numpy as np

from tqdm import tqdm

from alphadeep.model.featurize import \
    parse_aa_indices, \
    get_batch_mod_feature

from alphadeep._settings import \
    global_settings as settings, \
    const_settings

import alphadeep.model.base as model_base


mod_feature_size = len(const_settings['mod_elements'])

class TorchModelRT(torch.nn.Module):
    def __init__(self, 
        mod_feature_size,
        dropout=0.2
    ):
        super().__init__()
        BiRNN = True
        self.aa_embedding_size = 27
        hidden=128
        hidden_rnn_layer=2
        
        # ins_nce_embed_size = conf.max_instrument_num+1
        # self.instrument_nce_embed = torch.nn.Identity()
        
        output_hidden_size = hidden*(2 if BiRNN else 1)
        
        # mod_embed_size = 8
        # self.mod_embed_weights = torch.nn.Parameter(
            # torch.empty(mod_size, mod_embed_size), 
            # requires_grad = True
        # )
        self.dropout = torch.nn.Dropout(dropout)
        
        self.input = model_base.SeqLSTM(
            self.aa_embedding_size+mod_feature_size, 
            hidden, rnn_layer=1, 
            bidirectional=BiRNN
        )
        
        self.hidden = model_base.SeqLSTM(
            output_hidden_size, 
            hidden, rnn_layer=hidden_rnn_layer, 
            bidirectional=BiRNN
        )

        self.attn = model_base.SeqAttentionSum(
            output_hidden_size
        )

        self.output = model_base.LinearDecoder(
            output_hidden_size,
            1
        )

    def forward(self, 
        aa_indices, 
        mod_x,
    ):
        aa_x = torch.nn.functional.one_hot(
            aa_indices, self.aa_embedding_size
        )

        x = torch.cat((aa_x, mod_x), 2)
        x = self.input(x)
        x = self.dropout(x)
        
        x = self.hidden(x)
        x = self.dropout(x)

        x = self.attn(x)

        return self.output(x).squeeze(1)


In [4]:
#export
class AlphaRTModel(model_base.ModelImplBase):
    def __init__(self, dropout=0.2, lr=0.001):
        super().__init__()
        self.build(
            TorchModelRT, lr=lr, 
            dropout=dropout, 
            mod_feature_size=mod_feature_size
        )
        self.loss_func = torch.nn.L1Loss()
    
    def train(self, 
        precursor_df: pd.DataFrame,
        epoch=10,
        batch_size=1024,
        verbose=True
    ):
        self.model.train()

        for epoch in range(epoch):
            batch_cost = []
            _grouped = list(precursor_df.sample(frac=1).groupby('nAA'))
            rnd_nAA = np.random.permutation(len(_grouped))
            batch_tqdm = tqdm(rnd_nAA)
            for i_group in batch_tqdm:
                nAA, df_group = _grouped[i_group]
                df_group = df_group.reset_index(drop=True)
                for i in range(0, len(df_group), batch_size):
                    batch_end = i+batch_size-1 # DataFrame.loc[start:end] inlcudes the end

                    aa_indices = torch.LongTensor(
                        parse_aa_indices(
                            df_group.loc[i:batch_end, 'sequence'].values.astype('U')
                        )
                    )

                    mod_x_batch = get_batch_mod_feature(df_group.loc[i:batch_end,:], nAA)
                    mod_x = torch.Tensor(mod_x_batch)

                    RTs = torch.Tensor(df_group.loc[i:batch_end,'RT'].values)

                    cost = self._train_one_batch(
                        RTs,
                        aa_indices, mod_x
                    )
                    batch_cost.append(cost.item())
                batch_tqdm.set_description(
                    f'Epoch={epoch+1}, nAA={nAA}, Batch={len(batch_cost)}, Loss={cost.item():.4f}'
                )
            if verbose: print(f'[MS/MS training] epoch={epoch+1}, mean Loss={np.mean(batch_cost)}')

    def predict(self, precursor_df, batch_size=1024):
        self.model.eval()

        precursor_df['predict_RT'] = 0

        _grouped = precursor_df.groupby('nAA')
        df_list = []
        for nAA, df_group in tqdm(_grouped):
            df_group = df_group.reset_index(drop=True)
            for i in range(0, len(df_group), batch_size):
                batch_end = i+batch_size-1 # DataFrame.loc[start:end] inlcudes the end

                mod_x_batch = get_batch_mod_feature(df_group.loc[i:batch_end,:], nAA)

                aa_indices = torch.LongTensor(parse_aa_indices(
                    df_group.loc[i:batch_end, 'sequence'].values.astype('U')
                ))
                mod_x = torch.Tensor(mod_x_batch)

                predicts = self.model(
                    *[fea.to(self.device) for fea in
                    [aa_indices, mod_x]
                ]).cpu().detach().numpy()

                predicts[predicts<0] = 0

                df_group.loc[i:batch_end, 'predict_RT'] = predicts
            df_list.append(df_group)

        return pd.concat(df_list).reset_index(drop=True)

In [7]:
#export
def evaluate_RT(predict_df):
    import sklearn.metrics
    return pd.DataFrame({
        'R2':[sklearn.metrics.r2_score(predict_df['RT'], predict_df['predict_RT'])],
        'ExplainedVar': [sklearn.metrics.explained_variance_score(predict_df['RT'], predict_df['predict_RT'])],
        'MedianAE': [sklearn.metrics.median_absolute_error(predict_df['RT'], predict_df['predict_RT'])],
        'MeanAE': [sklearn.metrics.mean_absolute_error(predict_df['RT'], predict_df['predict_RT'])],
    })

In [8]:
evaluate_RT(pd.DataFrame({'RT':np.arange(10), 'predict_RT':np.arange(10)}))

,R2,ExplainedVar,MedianAE,MeanAE
0,1.0,1.0,0.0,0.0
